In [1]:
from transformers import InstructBlipProcessor, InstructBlipForConditionalGeneration
import torch
import os
import json
from PIL import Image
from tqdm import tqdm
# os.environ['CUDA_DEVICE_ORDER'] = 'PCI_BUS_ID'
# os.environ['CUDA_VISIBLE_DEVICES'] = '5,6,7'
device = "cuda:5"

/home/alsdyd98/anaconda3/envs/instblip/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# https://github.com/salesforce/LAVIS/tree/main/projects/instructblip#prepare-vicuna-weights
#!pip3 install "fschat[model_worker,webui]"
#!python3 -m fastchat.serve.cli --model-path lmsys/vicuna-7b-v1.5

In [3]:
from transformers import InstructBlipProcessor, InstructBlipForConditionalGeneration
import torch

processor = InstructBlipProcessor.from_pretrained("Salesforce/instructblip-vicuna-7b")
model = InstructBlipForConditionalGeneration.from_pretrained("Salesforce/instructblip-vicuna-7b", device_map = device, load_in_4bit=True, torch_dtype=torch.float16)

Loading checkpoint shards: 100%|██████████| 4/4 [01:05<00:00, 16.42s/it]


In [4]:
class AnswerGenerator:
    def __init__(self, model, preprocessor, device='cuda'):
        self.model = model
        self.preprocessor = preprocessor
        self.device = device

    @torch.no_grad()
    def _load(self, image, prompt):
        if type(image) == str:
            raw_image = Image.open(image).convert("RGB")
        else:
            raw_image = image
        # image_emb = self.preprocessor["eval"](raw_image).unsqueeze(0).to(device)
        inputs = processor(images=raw_image, text=prompt, return_tensors="pt").to(device=device, dtype=torch.float16)
        return inputs

    def inference(self, image, prompt):
        inputs = self._load(image, prompt)
        # qaSet = {"image": img_emb, "prompt": prompt}
        outputs = self.model.generate(**inputs,         
        num_beams=5,
        max_new_tokens=1024,
        min_length=1,
        top_p=0.9,
        repetition_penalty=1.5,
        length_penalty=1.0,
        temperature=1,)
        # print(processor.batch_decode(outputs, skip_special_tokens=True))
        generated_text = processor.batch_decode(outputs, skip_special_tokens=True)[0].strip()
        return generated_text


In [5]:
llm = AnswerGenerator(model, processor, device)
base_dir = "."
# key: imageFile_name, value: [{key(Q_i):value(question description)}]
with open(f'{base_dir}/question_data_updated.json', 'r') as f:
    question_dict = json.load(f)

answer_dict = {}
print(len(list(question_dict.keys())))
for fileName in tqdm(list(question_dict.keys())[160:]):
    answer_dict[fileName] = {}
    for q in question_dict[fileName]:
        # img_dir = f'{base_dir}/images/whoops_images/{fileName}.png'
        img_dir = f'{base_dir}/images/black_image.jpg'
        ans = llm.inference(img_dir, question_dict[fileName][q])
        ansKey = q
        answer_dict[fileName][ansKey] = ans
    with open(f'{base_dir}/answer_set_true_black_3.json', 'w') as outfile:
        json.dump(answer_dict, outfile)

with open(f'{base_dir}/answer_set_true_black_3.json', 'w') as outfile:
    json.dump(answer_dict, outfile)

230


  0%|          | 0/70 [00:00<?, ?it/s]/home/alsdyd98/anaconda3/envs/instblip/lib/python3.9/site-packages/bitsandbytes/nn/modules.py:226: UserWarning: Input type into Linear4bit is torch.float16, but bnb_4bit_compute_type=torch.float32 (default). This will lead to slow inference or training speed.
  warnings.warn(f'Input type into Linear4bit is torch.float16, but bnb_4bit_compute_type=torch.float32 (default). This will lead to slow inference or training speed.')
100%|██████████| 70/70 [2:16:51<00:00, 117.31s/it]
